# Determinant: Concept, Properties and Computation  
A concise refresher with NumPy / SymPy examples  
---

## 1. What is a determinant?
For every **square** matrix $A\in\mathbb{R}^{n\times n}$ there exists a scalar, denoted $\det(A)$ or $|A|$, that encodes:
- whether the matrix is invertible (non-zero determinant) or singular (zero determinant);  
- the product of all eigenvalues: $\det(A)=\prod_i\lambda_i$.

In [1]:
import numpy as np
A = np.array([[2, 1, 1],
              [1, 3, 2],
              [1, 0, 0]], dtype=float)
np.linalg.det(A)

np.float64(-1.0)

A non-zero determinant ⇒ $A$ is invertible.

## 2. Key properties (with quick NumPy checks)

### P1: $\det(I)=1$

In [2]:
np.linalg.det(np.eye(3))

np.float64(1.0)

### P2: Swapping two rows flips the sign

In [3]:
A_swap = A.copy()
A_swap[[0, 1]] = A_swap[[1, 0]]   # swap rows 0 and 1
np.linalg.det(A_swap), -np.linalg.det(A)

(np.float64(1.0), np.float64(1.0))

### P3: Determinant of a triangular matrix = product of diagonal entries

In [4]:
T = np.array([[2, 7, 3],
              [0, 5, 9],
              [0, 0, 4]], dtype=float)
np.linalg.det(T), 2*5*4

(np.float64(39.99999999999998), 40)

### P4: $\det(AB)=\det(A)\det(B)$

In [5]:
B = np.random.randint(-3, 3, (3, 3))
np.linalg.det(A @ B), np.linalg.det(A) * np.linalg.det(B)

(np.float64(0.0), np.float64(-0.0))

### P5: $\det(A^{T})=\det(A)$

In [6]:
np.allclose(np.linalg.det(A.T), np.linalg.det(A))

True

### P6: Row-scaling multiplies the determinant

In [7]:
k = 7
A_scaled = A.copy()
A_scaled[1] *= k               # scale row 1 by k
np.linalg.det(A_scaled), k * np.linalg.det(A)

(np.float64(-6.99999999999999), np.float64(-7.0))

### P7: Linearity in each row (or column)
If two matrices differ in **only one row**, say row $r$, then
$$\det(A) + \det(B) = \det(C)$$
where $C$ is identical to $A$ (and $B$) except that row $r$ of $C$ is the **element-wise sum** of row $r$ from $A$ and row $r$ from $B$.

In [8]:
import numpy as np

# build two 3×3 matrices that differ ONLY in row 1
A = np.array([[2, 1, 3],
              [4, 5, 6],
              [7, 8, 9]], dtype=float)

B = A.copy()
B[1] = np.array([10, 11, 12])

# matrix C whose row 1 = row1(A) + row1(B)
C = A.copy()
C[1] = A[1] + B[1]

lhs = np.linalg.det(A) + np.linalg.det(B)
rhs = np.linalg.det(C)
print(f"det(A) + det(B) = {lhs:.4f}")
print(f"det(C)          = {rhs:.4f}")
print(f"difference      = {abs(lhs - rhs):.2e}")

det(A) + det(B) = -0.0000
det(C)          = 0.0000
difference      = 2.31e-14


## 3. Computing determinants

### 3.1 Built-in 

In [9]:
# NumPy uses optimized LU factorization
np.linalg.det(A)

np.float64(-9.000000000000005)

### 3.2 Symbolic determinant with SymPy

In [10]:
import sympy as sp
sp.init_printing()
As = sp.Matrix(A)
As.det()

### 3.3 Manual cofactor expansion (educational, not efficient)

In [11]:
def cofactor_det(M):
    n = M.shape[0]
    if n == 1:
        return M[0, 0]
    det = 0.0
    for c in range(n):
        sub = np.delete(np.delete(M, 0, axis=0), c, axis=1)
        det += ((-1) ** c) * M[0, c] * cofactor_det(sub)
    return det

print(cofactor_det(A))

-9.0


### 3.4 Determinant by row-reduction (Gauss)

In [12]:
def det_by_rowops(A):
    M = A.astype(float).copy()
    n = M.shape[0]
    det_sign = 1.0
    for col in range(n):
        # partial pivot
        pivot = np.argmax(np.abs(M[col:, col])) + col
        if pivot != col:
            M[[col, pivot]] = M[[pivot, col]]
            det_sign *= -1
        if np.isclose(M[col, col], 0):
            return 0.0
        # elimination below
        for row in range(col + 1, n):
            factor = M[row, col] / M[col, col]
            M[row] -= factor * M[col]
    return det_sign * np.prod(np.diag(M))

det_by_rowops(A), np.linalg.det(A)

## 5. Quick cheat-sheet

| Task | One-liner |
|------|-----------|
| Determinant | `np.linalg.det(A)` |
| Singular? | `np.isclose(np.linalg.det(A), 0)` |
| Symbolic det | `sympy.Matrix(A).det()` |
